# 1. BERT 구현

In [ ]:
from transformers import AutoTokenizer, AutoModel # 토크나이저와 모델을 불러오기 위한 라이브러리 import
import torch # PyTorch 라이브러리 import

In [ ]:
# 사전학습된 BERT 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # 토크나이저 불러오기
model = AutoModel.from_pretrained("bert-base-uncased") # 모델 불러오기

# 추론 모드로 설정 (필수)
model.eval()

In [ ]:
# 입력 문장 (예시)
text = "The advancement of natural language processing has significantly improved human-computer interaction, enabling machines to understand context, emotion, and intention more effectively than ever before."

# 서브워드 문장 (예시)
# text = "The decentralization of decision-making is indispensable in hyperconnected environments."

In [ ]:
tokens = tokenizer.tokenize(text)
print(tokens)

In [ ]:
# 토큰화 → Tensor로 변환
inputs = tokenizer(text, return_tensors="pt")
print(inputs)

Inputs IDs (= Token ID)
* WordPiece 토큰을 정수 ID로 변환한 결과
* Token Embedding
* [101]: CLS 토큰
* [102]: SEP 토큰

In [ ]:
inputs.input_ids

token_type_ids
* 입력된 각 토큰이 어떤 문장에 속하는지를 표시하는 벡터
* Segment Embedding


In [ ]:
inputs.token_type_ids

attention_mask
* 각 토큰이 실제 단어인지(1) 또는 패딩인지(0)를 표시

In [ ]:
inputs.attention_mask

In [ ]:
text_a = "I love natural language processing."
text_b = "It helps computers understand human language."

inputs_2 = tokenizer(text_a, text_b, return_tensors="pt")
print(inputs_2)

In [ ]:
# 추론 (gradient 계산 X)
with torch.no_grad(): # PyTorch에서 “학습이 아닌 추론만 할 때” 사용하는 문법
    outputs = model(**inputs) # 토크나이저가 만든 입력을 BERT 모델에 전달하고, 결과를 받는 부분

outputs

last_hidden_state
* Tensor [batch_size, sequence_length, hidden_size]
* BERT 마지막 층의 출력 벡터 (12층 중 12층 결과)
* 각 토큰에 대한 contextual 임베딩
* [CLS] 토큰도 여기에 포함됨

In [ ]:
# 전체 hidden states (마지막 층 기준)
last_hidden_state = outputs.last_hidden_state
print(last_hidden_state.shape)  # (1, sequence_length, hidden_size)

[CLS] 토큰 벡터

In [ ]:
# [CLS] 위치의 벡터만 추출 (0번째 토큰)
cls_embedding = last_hidden_state[:, 0, :]  # Shape: (1, 768)
print(cls_embedding.shape)

pooler_output
* Tensor [batch_size, hidden_size]
* [CLS] 벡터에 선형 변환 + tanh 활성화를 적용한 결과
* 문장 분류 등에서 간단하게 문장 임베딩으로 사용
* [CLS] 벡터와 다름

In [ ]:
# 전체 hidden states (마지막 층 기준)
last_hidden_state = outputs.last_hidden_state
print(last_hidden_state.shape)  # (1, sequence_length, hidden_size)

# 2. BERT 임베딩 추출

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModel

from tqdm import tqdm # 진행 상황을 표시하기 위한 라이브러리 import
import pandas as pd # 데이터프레임을 사용하기 위한 라이브러리 import

In [ ]:
df= pd.read_csv("Musical_Instruments_2023_sampled.csv")
print(df.shape)
df.head()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
model.eval()

In [ ]:
def extract_cls_embedding(text_list):
    cls_embeddings = []

    with torch.no_grad():
        for text in tqdm(text_list):
            # 1. 텍스트 토큰화
            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            
            # 2. BERT에 입력 → 출력 벡터 얻기
            outputs = model(**inputs)
            last_hidden = outputs.last_hidden_state  # [1, seq_len, 768]
            
            # 3. [CLS] 벡터 추출 (첫 번째 토큰 위치)
            cls_vector = last_hidden[:, 0, :]  # shape: [1, 768]
            
            # 4. numpy 변환 및 리스트 저장
            cls_embeddings.append(cls_vector.squeeze().numpy())
    
    return cls_embeddings

In [ ]:
# 함수 실행
df['review_bert'] = extract_cls_embedding(df['text'])
df.head()

# 3. 예측 모델링

In [ ]:
import pandas as pd
import numpy as np
import random as python_random 
import tensorflow as tf
from sklearn.model_selection import train_test_split # 데이터셋을 학습/검증/테스트로 분할하기 위한 라이브러리 import

from tensorflow.keras.models import Model # 딥러닝 모델을 구성하기 위한 라이브러리 import
from tensorflow.keras.layers import Input, Dense, Dropout # 딥러닝 모델의 레이어를 구성하기 위한 라이브러리 import
from tensorflow.keras.optimizers import Adam # 딥러닝 모델의 최적화 알고리즘을 사용하기 위한 라이브러리 import
from tensorflow.keras.callbacks import EarlyStopping # 딥러닝 모델의 조기 종료를 위한 콜백 함수 import
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error # 회귀 모델의 성능 평가를 위한 라이브러리 import

def reset_random_seeds():
    tf.random.set_seed(42)
    np.random.seed(42)
    python_random.seed(42)
reset_random_seeds()

early_stopping = EarlyStopping(
    monitor='val_loss', # 모니터링할 지표
    patience=5,         # 해당 에포크 동안 개선되지 않으면 조기 종료
    verbose=1,          # 조기 종료 시 메시지 출력
    mode='auto',         # 모니터링할 지표의 개선 방향 설정(예측: min, 분류: max)
    restore_best_weights=True   # 최적 가중치 복원
)

## 데이터 분할

In [ ]:
def data_split(df_sample):
    df_train, df_test = train_test_split(df_sample, # 분할할 데이터셋
                                        test_size=0.2, # 테스트 데이터 비율
                                        random_state=42) # 랜덤 시드 설정

    train_bert = np.array(df_train['review_bert'].tolist()) # BERT 임베딩을 리스트로 변환
    y_train = np.array(df_train['rating']) # 평점 데이터를 NumPy 배열로 변환

    test_bert = np.array(df_test['review_bert'].tolist())
    y_test = np.array(df_test['rating'])

    return train_bert, test_bert, y_train, y_test

train_bert, test_bert, y_train, y_test = data_split(df)

## 예측 모델 구현

In [ ]:
def Rating_Predictor(learning_rate, dropout_rate):
# 1. Input

    bert_input = Input(shape=(768,), name='Review_BERT_Input') # BERT 임베딩 입력 레이어

    prediction = Dense(units=128, activation='relu', name='Pradiction_Layer_1')(bert_input) # 첫 번째 은닉층
    prediction = Dropout(rate=dropout_rate)(prediction) # 드롭아웃 레이어 추가
    prediction = Dense(units=64, activation='relu', name='Pradiction_Layer_2')(prediction) # 두 번째 은닉층
    prediction = Dropout(rate=dropout_rate)(prediction)
    prediction = Dense(units=32, activation='relu', name='Pradiction_Layer_3')(prediction) # 세 번째 은닉층
    prediction = Dropout(rate=dropout_rate)(prediction)

    prediction = Dense(1, activation='linear', name='Final_Layer')(prediction) # 최종 출력 레이어 (회귀용)

    # Model Definition
    model = Model(inputs=[bert_input], outputs=prediction) # 딥러닝 모델 정의

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mean_absolute_error', 'mean_squared_error']) # 모델 컴파일

    return model

In [ ]:
learning_rate = 0.0001 # 학습률 설정
batch_size = 32 # 배치 크기 설정
dropout_rate = 0.0 # 드롭아웃 비율 설정

model = Rating_Predictor(learning_rate, dropout_rate) # 딥러닝 모델 생성
model.summary() # 모델 요약 출력

## 학습

In [ ]:
model.fit([train_bert], y_train, # 모델 학습
        validation_split=0.125, # 검증 데이터 비율
        epochs=100,  # 에포크 수
        batch_size=batch_size, # 배치 크기
        callbacks=[early_stopping])  # 조기 종료 콜백 함수

## 예측

In [ ]:
predicted_ratings = model.predict([test_bert]) # 모델 예측
predicted_ratings

## 평가

In [ ]:
MAE_temp = mean_absolute_error(y_test, predicted_ratings) # MAE 계산
MSE_temp = mean_squared_error(y_test, predicted_ratings) # MSE 계산
RMSE_temp = np.sqrt(MSE_temp) # RMSE 계산
MAPE_temp = 100 * mean_absolute_percentage_error(y_test, predicted_ratings) # MAPE 계산

print(f'{MAE_temp:.4f}')
print(f'{MSE_temp:.4f}')
print(f'{RMSE_temp:.4f}')
print(f'{MAPE_temp:.4f}')